In [13]:
import google.generativeai as genai
import json
import time
import random
from google.api_core.exceptions import TooManyRequests

In [14]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY');
genai.configure(api_key=GOOGLE_API_KEY);

In [15]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image


In [16]:
small_model = genai.GenerativeModel('gemini-2.0-flash')
large_model = genai.GenerativeModel('gemini-2.5-flash')

In [17]:
questions = [
    { "id": 1, "question": "Otac kaže svom sinu: 'Kad sam ja imao tvoje godine, ti si imao 5 godina. Za tri godine, bit ću tri puta stariji nego što si ti sada.' Koliko godina trenutno imaju otac i sin?" },
    { "id": 2, "question": "U selu žive vitezovi (uvijek govore istinu) i lažljivci (uvijek lažu). A, B i C izjavljuju: A: 'B je lažljivac.' B: 'C je lažljivac.' C: 'A i B nisu iste vrste.' Tko je vitez, a tko lažljivac?" },
    { "id": 3, "question": "Imate 10 vrećica tableta. Sve teže 10 g osim jedne vrećice u kojoj su tablete mase 9 g. Imate pravo na samo jedno mjerenje. Kako otkriti koja je vrećica s lakšim tabletama?" },
    { "id": 4, "question": "Ako bi sve tvrdnje u ovoj rečenici bile neistinite, bi li tvrdnja 'barem jedna tvrdnja u ovoj rečenici je istinita' bila istinita?" },
    { "id": 5, "question": "Nijedan student koji nije zakasnio nije kažnjen, ali neki koji jesu zakasnili nisu kažnjeni. Koja tvrdnja mora biti istinita? A) Svi kažnjeni studenti zakasnili su. B) Svi koji su zakasnili su kažnjeni. C) Neki koji nisu zakasnili su kažnjeni. D) Neki koji su zakasnili su kažnjeni." },
    { "id": 6, "question": "Marko kaže: 'Ako skriveni broj pomnožim s 3, dodam 15 i podijelim s 6, dobit ću 10.' Koji je broj skriven?" },
    { "id": 7, "question": "U igri birate između 5 vrata, iza jednih je nagrada. Nakon vašeg izbora domaćin otvara 3 prazna vrata. Trebate li promijeniti izbor i kolika je vaša nova vjerojatnost dobitka ako promijenite?" },
    { "id": 8, "question": "Ispred dva vrata stoje iskreni i lažljivi čuvar. Smijete postaviti jedno pitanje jednom čuvaru. Koje pitanje treba postaviti da otkrijete vrata koja vode u slobodu?" },
    { "id": 9, "question": "Napiši posljednju riječ ove rečenice: 'Posljednja riječ ove rečenice nije ni prva ni druga nego _____.’ Koja mora biti posljednja riječ da je izjava istinita?" },
    { "id": 10, "question": "Sat kasni 10 minuta svakih 90 minuta. Nakon koliko stvarnih sati će pokazivati točno 12 sati ukupnog kašnjenja?" }
]


In [18]:
answers = [
    { "id": 1, "question": "Otac kaže svom sinu: 'Kad sam ja imao tvoje godine, ti si imao 5 godina. Za tri godine, bit ću tri puta stariji nego što si ti sada.' Koliko godina trenutno imaju otac i sin?", "answer": "Sin ima 7 godina, otac 28 godina." },
    { "id": 2, "question": "U selu žive vitezovi i lažljivci... A: 'B je lažljivac.' B: 'C je lažljivac.' C: 'A i B nisu iste vrste.' Tko je tko?", "answer": "A je vitez, B je lažljivac, C je vitez." },
    { "id": 3, "question": "Imate 10 vrećica tableta... jedno mjerenje...", "answer": "Uzmite iz prve vrećice 1 tabletu, iz druge 2, iz treće 3 itd. Razlika od očekivane težine pokazuje broj vrećice s lakšim tabletama." },
    { "id": 4, "question": "Ako bi sve tvrdnje u ovoj rečenici bile neistinite...", "answer": "Ne bi mogla biti istinita; rečenica je paradoks i ne može biti konzistentno istinita niti potpuno neistina." },
    { "id": 5, "question": "Nijedan student koji nije zakasnio nije kažnjen... koja tvrdnja mora biti istinita?", "answer": "D) Neki studenti koji su zakasnili su kažnjeni." },
    { "id": 6, "question": "Marko kaže... 'dobijem 10'. Koji je broj?", "answer": "Skriveni broj je 15." },
    { "id": 7, "question": "Birate između 5 vrata... domaćin otvara 3...", "answer": "Da, treba promijeniti. Vjerojatnost dobitka ako promijenite je 4/5 = 80%." },
    { "id": 8, "question": "Jedno pitanje jednom čuvaru da otkrijete prava vrata...", "answer": "Pitajte: 'Koja bi vrata drugi čuvar rekao da vode u smrt?' Onda odaberite suprotna." },
    { "id": 9, "question": "Posljednja riječ ove rečenice nije ni prva ni druga nego _____.", "answer": "Točna riječ je: 'treća'." },
    { "id": 10, "question": "Sat kasni 10 minuta svakih 90 minuta... kada će kasniti ukupno 12 sati?", "answer": "Nakon 72 sata stvarnog vremena." }
  ]


In [19]:
def call_with_retry(func, max_retries=5):
    """Poziv funkcije s retry logikom za 429 greške"""
    for attempt in range(max_retries):
        try:
            return func()
        except TooManyRequests as e:
            wait_time = getattr(e, 'retry_after', (2 ** attempt + random.random()))
            print(f"Quota exceeded, retrying in {wait_time:.2f}s...")
            time.sleep(wait_time)
    raise Exception("Max retries exceeded")

In [20]:
def get_small_model_answers(prompt_text):
    """Šalje prompt malom modelu i vraća odgovore"""
    questions_text = "\n".join([f"{q['id']}. {q['question']}" for q in questions])
    full_prompt = f"{prompt_text}\n\n{questions_text}"

    def _call():
        response = small_model.generate_content(full_prompt)
        return response.text

    return call_with_retry(_call)

In [21]:
def evaluate_answers(small_model_response):
    """Evaluira odgovore malog modela koristeći veliki model"""
    eval_prompt = f"""Evaluiraj sljedeće odgovore modela u odnosu na točne odgovore.
Odgovor ne mora biti identičan, već imati istu ideju/značenje.

ODGOVORI MODELA:
{small_model_response}

TOČNI ODGOVORI:
{chr(10).join([f"{a['id']}. {a['question']} -> {a['answer']}" for a in answers])}

Vrati samo broj točnih odgovora u formatu: "SCORE: X/10" gdje je X broj točnih odgovora."""

    def _call():
        response = large_model.generate_content(eval_prompt)
        return response.text

    result = call_with_retry(_call)

    # Parsiranje rezultata
    try:
        score_line = [line for line in result.split('\n') if 'SCORE:' in line][0]
        score = int(score_line.split(':')[1].strip().split('/')[0])
        return score
    except:
        print(f"Nije moguće parsirati score, puni odgovor:\n{result}")
        return 0


In [22]:
def correct_prompt_with_large_model(original_prompt, small_model_response):
    """Koristi veliki model za poboljšanje prompta"""
    correction_prompt = f"""Imaš sljedeći prompt i odgovore koje je model dao:

PROMPT:
{original_prompt}

ODGOVORI MODELA:
{small_model_response}

Tvoj zadatak je poboljšati ovaj prompt kako bi model dao bolje odgovore.
Vrati samo poboljšani prompt, bez dodatnih objašnjenja. Pazi da ne mijenjaš ideju pitanja i da ne dodavaš dodatne informacije ili unutar pitanja, ali ga uputi da na neki drugi način pristupi preoblemu"""

    def _call():
        response = large_model.generate_content(correction_prompt)
        return response.text

    return call_with_retry(_call)


In [23]:
def main():
    current_prompt = "Riješi ova pitanja:"
    max_iterations = 10

    print("=== POČETAK OPTIMIZACIJE PROMPTA ===\n")

    for iteration in range(1, max_iterations + 1):
        print(f"\n--- ITERACIJA {iteration} ---")
        print(f"Trenutni prompt: {current_prompt[:100]}...")

        # 1. Dobij odgovore od malog modela
        print("\n[1] Pozivam mali model...")
        small_model_response = get_small_model_answers(current_prompt)
        print(f"Odgovor malog modela (prvih 200 znakova):\n{small_model_response[:200]}...\n")

        # 2. Evaluiraj odgovore
        print("[2] Evaluiram odgovore...")
        score = evaluate_answers(small_model_response)
        print(f"SCORE: {score}/10")

        # 3. Provjeri uvjete za prekid
        if score == 10:
            print(f"\n🎉 USPJEH! Postignut perfektan score u {iteration}. iteraciji!")
            print(f"Finalni prompt:\n{current_prompt}")
            break

        if iteration == max_iterations:
            print(f"\n⚠️ Dostignut maksimalan broj iteracija ({max_iterations})")
            print(f"Finalni score: {score}/10")
            print(f"Finalni prompt:\n{current_prompt}")
            break

        # 4. Poboljšaj prompt
        print("[3] Poboljšavam prompt...")
        current_prompt = correct_prompt_with_large_model(current_prompt, small_model_response)
        print(f"Novi prompt: {current_prompt[:100]}...")

    print("\n=== KRAJ OPTIMIZACIJE ===")

# Pokreni glavnu petlju
main()

=== POČETAK OPTIMIZACIJE PROMPTA ===


--- ITERACIJA 1 ---
Trenutni prompt: Riješi ova pitanja:...

[1] Pozivam mali model...
Odgovor malog modela (prvih 200 znakova):
Evo rješenja za svako pitanje:

1.  **Starost oca i sina:**

    *   Neka je očeva trenutna starost *O*, a sinova *S*.
    *   Iz prve rečenice:  *O - (S - 5) = S*  (Očeva starost prije S-5 godina bil...

[2] Evaluiram odgovore...
SCORE: 6/10
[3] Poboljšavam prompt...
Novi prompt: Riješi sljedeća pitanja prateći ove upute za svaki odgovor:

1.  **Ponavljanje pitanja i pretpostavk...

--- ITERACIJA 2 ---
Trenutni prompt: Riješi sljedeća pitanja prateći ove upute za svaki odgovor:

1.  **Ponavljanje pitanja i pretpostavk...

[1] Pozivam mali model...
Odgovor malog modela (prvih 200 znakova):
## Rješavanje problema korak po korak

Slijedit ću upute i za svaki problem ću:

1.  Ponavljati pitanje i pretpostavke.
2.  Definirati elemente.
3.  Navoditi sistematičan postupak rješavanja.
4.  Rješ...

[2] Evaluiram odgovore...
SCOR